## 1. 라이브러리 불러오기

In [1]:
import sys
import glob
import cv2
import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils, datasets, models
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

from torch.autograd import Variable

from matplotlib import pyplot as plt
from time import time

import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold

import wandb

from augraphy import *

In [2]:
meta_path = '/data/ephemeral/home/data/meta.csv'
train_path = '/data/ephemeral/home/data/train.csv'
submission_path = '/data/ephemeral/home/data/sample_submission.csv'

meta_data = pd.read_csv(meta_path)
df_train = pd.read_csv(train_path)
df_submission = pd.read_csv(submission_path)

merge = pd.merge(df_train, meta_data, how='inner')

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

## 2. Custom Dataset

In [4]:
class ImageDataset(Dataset):
    def __init__(self, csv, path, album_transform=None, augraphy_transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path 
        self.album_transform = album_transform
        self.augraphy_transform = augraphy_transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        
        if self.augraphy_transform:
            img = self.augraphy_transform(img)

        if self.album_transform:
            img = self.album_transform(image=img)['image']
        
        return img, target

## 3. Training Pipeline

In [5]:
def training(model, dataloader, dataset, device, criterion, optimizer, epoch, num_epochs):
    model.train()
    train_loss = 0.0
    preds_list = []
    targets_list = []

    tbar = tqdm(dataloader)
    for images, labels in tbar:
        images = images.type(torch.cuda.FloatTensor)
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(outputs.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(labels.detach().cpu().numpy())

        tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Train Loss : {loss.item():.4f}")

    train_loss = train_loss / (len(dataloader))
    train_acc = accuracy_score(preds_list, targets_list)
    train_f1 = f1_score(preds_list, targets_list, average='macro')

    metrics = {
        'train_loss' : train_loss,
        'train_acc' : train_acc,
        'train_f1' : train_f1
    }

    return model, metrics

def evaluation(model, dataloader, dataset, device, criterion, epoch, num_epochs):
    model.eval()
    valid_loss = 0.0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        tbar = tqdm(dataloader)
        for images, labels in tbar:
            images = images.type(torch.cuda.FloatTensor)
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
            preds_list.extend(outputs.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(labels.detach().cpu().numpy())

            tbar.set_description(f"Epcoh [{epoch+1}/{num_epochs}] Valid Loss : {valid_loss:.4f}")

    valid_loss /= len(dataloader)
    valid_acc = accuracy_score(preds_list, targets_list)
    valid_f1 = f1_score(preds_list, targets_list, average='macro')

    metrics = {
        'valid_loss' : valid_loss,
        'valid_acc' : valid_acc,
        'valid_f1' : valid_f1
    }

    return model, metrics

def training_loop(model, train_dataloader, valid_dataloader, train_dataset, valid_dataset, criterion, optimizer, device, num_epochs, model_path, model_name, patience, run):

    best_valid_loss = float('inf')
    valid_max_accuracy = -1
    valid_max_f1 = -1
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model, train_metrics = training(model, train_dataloader, train_dataset, device, criterion, optimizer, epoch, num_epochs)
        model, valid_metrics = evaluation(model, valid_dataloader, valid_dataset, device, criterion, epoch, num_epochs)

        monitoring_value = {
            'train_loss' : train_metrics['train_loss'],
            'train_accuracy' : train_metrics['train_acc'],
            'train_f1' : train_metrics['train_f1'],
            'valid_loss' : valid_metrics['valid_loss'],
            'valid_accuracy' : valid_metrics['valid_acc'],
            'valid_f1' : valid_metrics['valid_f1']
        }
        run.log(monitoring_value, step=epoch)

        if valid_max_accuracy < valid_metrics['valid_acc']:
            valid_max_accuracy = valid_metrics['valid_acc']

            run.summary['best_train_acc'] = train_metrics['train_acc']
            run.summary['best_valid_acc'] = valid_metrics['valid_acc']
        
        if valid_max_f1 < valid_metrics['valid_f1']:
            valid_max_f1 = valid_metrics['valid_f1']
            torch.save(model.state_dict(), model_path+f"/model_{model_name}.pt")

            run.summary['best_train_f1'] = train_metrics['train_f1']
            run.summary['best_valid_f1'] = valid_metrics['valid_f1']

        if best_valid_loss > valid_metrics['valid_loss']:
            best_valid_loss = valid_metrics['valid_loss']
            early_stop_counter = 0
            run.summary['best_train_loss'] = train_metrics['train_loss']
            run.summary['best_valid_loss'] = valid_metrics['valid_loss']
        else:
            early_stop_counter += 1
            
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss : {train_metrics['train_loss']:.4f}, Train Acc : {train_metrics['train_acc']:.4f}, 'Train F1 : {train_metrics['train_f1']:.4f}, Valid Loss : {valid_metrics['valid_loss']:.4f}, Valid Acc : {valid_metrics['valid_acc']:.4f}, Valid F1 : {valid_metrics['valid_f1']}")

        if early_stop_counter >= patience:
            print('Early Stopping!')        
            break

    return model, valid_max_accuracy, valid_max_f1


## 4. Data Load

In [6]:
img_csv_path = 'data/aug_dataset/aug_data.csv'
df_img = pd.read_csv(img_csv_path)
df_img.head()

,ID,target
0,augmented_0_a1ab865095b2d312_ljh.jpg,2
1,augmented_1_a1ab865095b2d312_ljh.jpg,2
2,augmented_2_a1ab865095b2d312_ljh.jpg,2
3,augmented_3_a1ab865095b2d312_ljh.jpg,2
4,augmented_4_a1ab865095b2d312_ljh.jpg,2


In [7]:
img_path = 'data/aug_dataset/aug_2'
test_img_path = '/data/ephemeral/home/data/test/'
totensor_transform = A.Compose([A.Resize(380, 380), ToTensorV2()])
test_transform = A.Compose([
    A.Resize(380, 380),
    ToTensorV2()
])

train_dataset = ImageDataset(img_csv_path, img_path, album_transform=totensor_transform, augraphy_transform=None)
test_dataset = ImageDataset(submission_path, test_img_path, album_transform=test_transform, augraphy_transform=None)

print(len(train_dataset), len(test_dataset))

23550 3140


In [8]:
train_num, valid_num = int(len(train_dataset) * 0.8), int(len(train_dataset) * 0.2)
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_num, valid_num])

print(len(train_dataset), len(valid_dataset))

18840 4710


In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 5. Train Model

In [10]:
model = timm.create_model('efficientnet_b4', pretrained=True)
in_features = model.classifier.in_features
classifier = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(256, 17),
) 

model.classifier = classifier

### Hyper Parameter 정의

In [11]:
class Cfg():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100
    batch_size=32
    model_path = '/data/ephemeral/home/models'

In [12]:
run = wandb.init(project='AIStage-CV', name='effb4_add_fc')

device = Cfg.device
model = Cfg.model
criterion = Cfg.criterion
optimizer = Cfg.optimizer 
num_epochs = Cfg.num_epochs
model_name = 'effb4-add_fc'
model_path = Cfg.model_path

run.watch(model, criterion, log='all', log_graph=True)

model, valid_max_accuracy, valid_max_f1 = training_loop(model, train_dataloader, valid_dataloader, train_dataset, valid_dataset, criterion, optimizer, device, num_epochs, model_path, model_name, 20, run)

run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: myeongwang (pluto_0905). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
Epcoh [1/100] Valid Loss : 33.0519: 100%|██████████| 148/148 [00:30<00:00,  4.82it/s]


Epoch [1/100], Train Loss : 0.4942, Train Acc : 0.8325, 'Train F1 : 0.8172, Valid Loss : 0.2233, Valid Acc : 0.9153, Valid F1 : 0.9112990052198747


Epcoh [2/100] Valid Loss : 23.9640: 100%|██████████| 148/148 [00:29<00:00,  4.97it/s]


Epoch [2/100], Train Loss : 0.1751, Train Acc : 0.9385, 'Train F1 : 0.9349, Valid Loss : 0.1619, Valid Acc : 0.9393, Valid F1 : 0.9366239606898


...

Epoch [50/100], Train Loss : 0.0061, Train Acc : 0.9985, 'Train F1 : 0.9985, Valid Loss : 0.0493, Valid Acc : 0.9864, Valid F1 : 0.9854826569681547


Epcoh [51/100] Valid Loss : 8.0554: 100%|██████████| 148/148 [00:29<00:00,  4.95it/s]


Epoch [51/100], Train Loss : 0.0149, Train Acc : 0.9961, 'Train F1 : 0.9960, Valid Loss : 0.0544, Valid Acc : 0.9837, Valid F1 : 0.9822088842690073


Epoch [52/100], Train Loss : 0.0000:  34%|███▍      | 199/589 [01:11<02:20,  2.78it/s]


KeyboardInterrupt: 

In [13]:
# run = wandb.init(project='AIStage-CV', name='effb4_add_fc')

# device = Cfg.device
# model = Cfg.model
# criterion = Cfg.criterion
# optimizer = Cfg.optimizer 
# num_epochs = Cfg.num_epochs
# model_name = 'effb4-add_fc'
# model_path = Cfg.model_path

# run.watch(model, criterion, log='all', log_graph=True)

# model, valid_max_accuracy, valid_max_f1 = training_loop(model, train_dataloader, valid_dataloader, train_dataset, valid_dataset, criterion, optimizer, device, num_epochs, model_path, model_name, 20, run)

# run.finish()

In [14]:
effb4 = timm.create_model('efficientnet_b4', pretrained=True)
in_features = effb4.classifier.in_features
classifier = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.SiLU(), # relu -> swish 변경 
    nn.Dropout(p=0.2),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(256, 17),
)

effb4.classifier = classifier
effb4.load_state_dict(torch.load('/data/ephemeral/home/models/model_effb4-add_fc.pt'))
effb4 = effb4.to(device)
effb4.eval()

EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [15]:
preds_list = []

for images, labels in tqdm(test_dataloader):
    images = images.type(torch.cuda.FloatTensor)
    images = images.to(device)

    with torch.no_grad():
        preds = effb4(images)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:29<00:00,  3.31it/s]


In [16]:
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [17]:
sample_submission_df = pd.read_csv(submission_path)
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [18]:
pred_df.to_csv('/data/ephemeral/home/outputs/effb4-add_fc.csv', index=False)

In [19]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
